# Лекция 9: кодировки и Python

In [1]:
def process_line(line):
    line = unicode(line)
    return line.split(' ')[1]

line = "Hello, привет!\n"
print process_line(line)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xd0 in position 7: ordinal not in range(128)

## Кодировки и стандарт Unicode

### Что такое кодировки и как появились

#### Что такое кодировка?

* В компьютере данные представлены в виде набора бит объединённых в байты.
* Биты и байты можно можно использовать для записи текста.
* Формат представления текста в бинарном виде называется кодировкой.

#### ASCII

* American Standard Code for Information
* 1968 год
* Определяла коды для символов, от 0 до 127.
* В основном состояла из строчных и прописных символов английского алфавита, цифр, пунктуации, математических символов и специальных управляющих кодов.
* 'a' = 97
* "a..z" (и "A..Z") идут подряд (не во всех кодировках так)

#### Локальные альтернативы ascii
* Американский стандарт, не было нужных символов с диакритикой (французский) или просто символов алфавита (русский). Какое-то время с этим мирились.
* Некоторые компании создавали свои альтернативы, но они не были общеприняты.
* В 1980 большинство персональных компьютеров имели 8-битные байты, поэтому можно было хранить значения 0..255.
* Альтернативные кодировки дополняли ascii дописывая используя вторую половину байта.
* Примеры: для русского (koi8-r, cp1251), французского (latin1) и т.д. 

#### Проблемы с однобайтовыми кодировками

* Сложно использовать несколько языков одновременно.
* Нетривиальность определения кодировок при совместном использовании (поэтому обычно использовали одну).
* Неправильное отображение при неправильном определении.
* Неуниверсальность записи различных данных.

### Стандарт Unicode

#####  Начало стандартизации

* В 1980-х люди хотели решить проблемы связанные с использованием однобайтных кодировок.
* Началось движение в сторону стандартизации общего Unicode формата представления и записи символов.
* Сначала хотели 16-битные символы, но оказалось мало.
* Решили формально не ограничивать пространство кодов (но по факту сейчас все символы помещаются в 4 байта и расширять пока не планируется).

##### Две основные части стандарта

* UCS (universal character set) - универсальный набор символов Unicode, задано соответствие между символом и кодом.
* UTF (unicode transformation format) - семейство кодировок определяет машинное представление последовательности кодов UCS.

##### Определения
* Символ (character) - наименьшая единица текста ('A', 'È', 'Ω' и ...).
* Точка кода (code point) - целое число из таблицы отображения символов на коды, представляет отдельный символ.
* Глиф (glyph) - графические элементы записи конкретного символа (обычно обрабатываются прозрачно для нас, заботу на себя берёт ОС, браузер, GUI).
* Юникод (Unicode) - стандарт кодирования символов, по сути представляет собой таблицу с описаниями символов, правила и рекомендации их представления и использования. 
* Кодировка - правила перевода символов Unicode в последовательность байт (UTF часть стандарта).

##### Детальнее про внутреннее представление

https://ru.wikipedia.org/wiki/Юникод

#### Кодировки и Unicode

* Кодировки могут быть постоянного и переменного размера.
* Постоянного:
  * latin1 (однобайтовая, не юникод)
  * UCS-2 (двухбайтовая; аналог UTF-16 без суррогатных пар)
  * UCS-4 (четырёхбайтовая, синоним UTF-32).
* Переменного:
  * UTF-8

#### UTF-16 и UTF-32

* 2 и 4 байтовые юникод кодировки.
* Фиксированной длинны.
* Важен порядок байт: BE (big-endian) или LE (little-endian).
* В начале файлов часто бывает символ BOM - byte order mark - обозначающий BE или LE.
* Из-за BOM могут возникать проблемы. Важно понимать, когда он есть и когда его убирать.
* А лучше использовать UTF-8.

#### UTF-8

##### Правила записи UTF-8

* Если код меньше 128, то просто записываем байт.
* Если код между 128 и 0x7ff, то записываем два байта со значениями между 128 и 255.
* Для кодов больше 0x7ff используются трёх и четырёх байтовые последовательности, каждый байт со значениями между 128 и 255.
* Теоретически, расширяется до 6 байт, но на практике (в текущем Unicode) хватает только 4 байт, т.к. нет символов с кодом больше 0x10FFFF).

Unicode UTF-8:
* 0x00000000 — 0x0000007F: 0xxxxxxx
* 0x00000080 — 0x000007FF: 110xxxxx 10xxxxxx
* 0x00000800 — 0x0000FFFF: 1110xxxx 10xxxxxx 10xxxxxx
* 0x00010000 — 0x001FFFFF: 11110xxx 10xxxxxx 10xxxxxx 10xxxxxx

Теоретически возможны, но не включены в стандарт также:

* 0x00200000 — 0x03FFFFFF: 111110xx 10xxxxxx 10xxxxxx 10xxxxxx 10xxxxxx
* 0x04000000 — 0x7FFFFFFF: 1111110x 10xxxxxx 10xxxxxx 10xxxxxx 10xxxxxx 10xxxxxx

Замечания:
* Несмотря на то, что UTF-8 позволяет указать один и тот же символ несколькими способами, только наиболее короткий из них правильный.
* Остальные формы должны отвергаться по соображениям безопасности.

##### Удобные свойства UTF-8
* Можно записать любой существующий символ Unicode.
* В записях utf-8 символов нет промежуточных нулевых байт, поэтому такие строки совместимы с C-функциями вроде strcpy и т.д., а также могут передаваться по протоколам не поддерживающим промежуточные нулевые байты.
* Строка ASCII - валидный UTF-8 текст.
* UTF-8 достаточно компактен: большинство используемых символов помещаются в 1-2 байтовые последовательности.
* В случае повреждения или потери данных можно определить начало следующего UTF-8 кода и продолжить с этого места (+ маловероятно, что случайные данные будут выглядеть как UTF-8).

## Работа с кодировками в Python

Работам с кодировками когда:
* не английский язык
* сторонние библиотеки
* обработка произвольного текстового ввода-вывода

### str и unicode

* Общий предок: basestring
* str - однобайтовые строки
* unicode - юникод-строки (представление зависит от интерпретатора, 16 или 32 битные целые).

#### unicode
* unicode(string[, encoding, errors])
* Все аргументы - 8-битные строки.

In [2]:
unicode('abcdef')

u'abcdef'

In [3]:
s = unicode('abcdef')

In [4]:
type(s)

unicode

In [5]:
unicode('abcdef' + chr(255))

UnicodeDecodeError: 'ascii' codec can't decode byte 0xff in position 6: ordinal not in range(128)

Аргумент errors определяет поведение при ошибке:
* 'strict' - (выбран по-умолчанию) выбрасывать исключение UnicodeDecodeError.
* 'replace' - замена проблемных символов на U+FFFD (‘REPLACEMENT CHARACTER’).
* 'ignore' - пропустить проблемный символ, не учитывать его в результирующей юникод записи.

In [6]:
unicode('\x80abc', errors='strict')  

UnicodeDecodeError: 'ascii' codec can't decode byte 0x80 in position 0: ordinal not in range(128)

In [7]:
unicode('\x80abc', errors='replace')

u'\ufffdabc'

In [8]:
unicode('\x80abc', errors='ignore')

u'abc'

Можно создавать односимвольные строки:

In [9]:
unichr(40960)

u'\ua000'

И обратно:

In [10]:
ord(u'\ua000')

40960

### Кодирование и декодирование

* Кодировка в Python задаётся своим названием (таблица встроенных кодировок https://docs.python.org/2/library/codecs.html#standard-encodings).
* Методы encode (и decode) используют имя кодировки для явного указания целевой кодировки (кодировки источника).

#### .encode([encoding], [errors='strict'])

Конвертация unicode() строки в str() с указанной кодировкой encoding.

In [11]:
u = unichr(40960) + u'abcd' + unichr(1972)
print u

ꀀabcd޴


In [12]:
u.encode('utf-8')

'\xea\x80\x80abcd\xde\xb4'

In [13]:
print [bin(ord(elem)) for elem in u.encode('utf-8')]

['0b11101010', '0b10000000', '0b10000000', '0b1100001', '0b1100010', '0b1100011', '0b1100100', '0b11011110', '0b10110100']


In [14]:
u.encode('ascii')

UnicodeEncodeError: 'ascii' codec can't encode character u'\ua000' in position 0: ordinal not in range(128)

In [15]:
u.encode('ascii', 'ignore')

'abcd'

In [16]:
u.encode('ascii', 'replace')

'?abcd?'

In [17]:
u.encode('ascii', 'xmlcharrefreplace')

'&#40960;abcd&#1972;'

#### .decode([encoding], [errors])

Конвертация str() строки в unicode() предполагая, что str() имеет кодировку encoding.

In [18]:
u = unichr(40960) + u'abcd' + unichr(1972)   # Assemble a string

In [19]:
utf8_version = u.encode('utf-8')             # Encode as UTF-8

In [20]:
type(utf8_version), utf8_version

(str, '\xea\x80\x80abcd\xde\xb4')

In [21]:
u2 = utf8_version.decode('utf-8')            # Decode using UTF-8

In [22]:
u == u2                                # The two strings match

True

#### Низкоуровневые преобразования и работа с кодировками

* Можно найти в модуле codecs.
* Обычно, это не нужно и базовых достаточно.

### unicode literals

In [23]:
s = u"a\xac\u1234\u20ac\U00008000"

* \x - два hex.
* \u - четыре hex.
* \U - восемь hex.

In [24]:
for character in s:
    print ord(character),

97 172 4660 8364 32768


### -*- coding: utf-8 -*-

* Кодировка в Python 2 по-умолчанию - ASCII (в версии до 2.4 использовалась latin-1).
* Чтобы упростить запись спецсимволов в файле с исходным кодом, можно задать другую кодировку.
* Принято использовать соглашение, взятое из emacs.

In [25]:
# -*- coding: utf-8 -*-

Будет работать и с "coding: name" или "coding=name".

### codecs

* Иногда необходима низкоуровневая работа с байтами кодировок (например, если оборвана последовательность байт).
* Для этого можно воспользоваться модулем codecs.
* В частности, с помощью codecs.open можно явно указывать кодировку открываемого файла и правила обработки ошибок.

codecs.open(filename, mode='rb', encoding=None, errors='strict', buffering=1)

In [26]:
import codecs
with codecs.open('codecs_test', encoding='utf-8', mode='w+') as f:
    f.write(u'\u4500 blah blah blah\n')
    f.seek(0)
    print repr(f.readline()[:1]) # U+FEFF byte order mark

u'\u4500'


#### BOM

In [27]:
import codecs
print repr(codecs.BOM_UTF16_LE)
print repr(codecs.BOM_UTF16_BE)
print repr(codecs.BOM_UTF8) # has no big sense for UTF-8

'\xff\xfe'
'\xfe\xff'
'\xef\xbb\xbf'


Про удаление BOM:
* Декодирование из UTF-16 удаляет BOM автоматически.
* Но не из UTF-8: надо явно использовать s.decode('utf-8-sig')

### Свойства Unicode символов

In [28]:
import unicodedata

u = unichr(233) + unichr(0x0bf2) + unichr(3972) + unichr(6000) + unichr(13231)

for i, c in enumerate(u):
    print i, '%04x' % ord(c), unicodedata.category(c),
    print unicodedata.name(c)

# Get numeric value of second character
print unicodedata.numeric(u[1])

0 00e9 Ll LATIN SMALL LETTER E WITH ACUTE
1 0bf2 No TAMIL NUMBER ONE THOUSAND
2 0f84 Mn TIBETAN MARK HALANTA
3 1770 Lo TAGBANWA LETTER SA
4 33af So SQUARE RAD OVER S SQUARED
1000.0


* Категории: Letter, Number, Punctuation, Symbol
* Подкатегории зависят от категорий:
  * Ll (letter, lowercase)
  * No (number, other)
  * Mn (mark, nonspacing)
  * So (symbol, other)

### Общие проблемы при работе с кодировками в Python

* ASCII является кодировкой по-умолчанию в Python 2.
* Файлы могут содержать BOM (byte order mark).
* Не все внутренние части Python 2 поддерживают юникод.
* Невозможно точно угадать кодировку.
* Чтение чанками многобайтовых кодировок.

### Общие советы по работе с кодировками

#### Декодируем, обрабатываем, кодируем

* Декодируем входные данные как можно раньше на входе
* Используем внутри своей программы Unicode.
* Кодируем только в самом конце перед передачей данных на выход программы.

#### Обертки для модулей

Можно писать обертки для модулей неподдерживающих юникод.

#### Юникод и тесты

Помещать проверки работы с юникодом в юнит тесты.

#### Угадывание и UTF-8 ftw

* По-умолчанию лучше предполагать (и использовать самому) utf-8.
* Дополнительно можно использовать byte order mark для угадывания кодировки.
* chardet.detect() от http://chardet.feedparser.org/

## utf-8 playground

In [29]:
def print_utf8_repr(unistr):
    utf8_line = unistr.encode('utf-8')
    for character in utf8_line:
        print bin(ord(character))[2:].zfill(8),

In [30]:
print_utf8_repr(unichr(127))

01111111


In [31]:
print_utf8_repr(unichr(128))

11000010 10000000


In [32]:
print_utf8_repr(unichr(4095))

11100000 10111111 10111111


In [33]:
print_utf8_repr(unichr(2048))

11100000 10100000 10000000


In [34]:
print_utf8_repr(unichr(2047))

11011111 10111111


In [35]:
print_utf8_repr(unichr(131071))

11110000 10011111 10111111 10111111


In [36]:
print_utf8_repr(unichr(65535))

11101111 10111111 10111111


In [37]:
print_utf8_repr(unichr(65536))

11110000 10010000 10000000 10000000


0  
  
110 10  
1110 10  
11110 10  

#### Самостоятельная задача

Рекомендую попробовать написать функцию кодирования (декодирования) utf-8.

## Источники и детальнее

* https://ru.wikipedia.org/wiki/Юникод
* https://docs.python.org/2/howto/unicode.html
* https://docs.python.org/2/library/codecs.html#standard-encodings
* http://farmdev.com/talks/unicode/
* https://pythonhosted.org/kitchen/unicode-frustrations.html